In [94]:
import getpass
from sqlalchemy import Column, Integer, String, ForeignKey, Date
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
import datetime

In [95]:
%run ./engine.ipynb

In [36]:
Base = declarative_base()

In [37]:
class Author(Base):
    __tablename__ = "authors"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(100), nullable=False)
    books = relationship("Book", back_populates="author")


class Book(Base):
    __tablename__ = "books"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    title = Column(String(200), nullable=False)
    author_id = Column(Integer, ForeignKey("authors.id"), nullable=True)
    genre=Column(String(50),nullable=True)
    copies_total = Column(Integer, default=1)             
    copies_available = Column(Integer, default=1)
    publisher = Column(String(100), nullable=True)
    language = Column(String(50), nullable=True)
    author = relationship("Author", back_populates="books")
    loans = relationship("Loan", back_populates="book")
class Borrower(Base):
    __tablename__ = "borrowers"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(100), nullable=False)
    email = Column(String(100), unique=True, nullable=False) 
    phone = Column(String(20), nullable=True)           
    membership_date = Column(Date, default=datetime.date.today) 
    address = Column(String(200), nullable=True)      
    loans = relationship("Loan", back_populates="borrower")


class Loan(Base):
    __tablename__ = "loans"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    book_id = Column(Integer, ForeignKey("books.id"), nullable=False)
    borrower_id = Column(Integer, ForeignKey("borrowers.id"), nullable=False)
    loan_date = Column(Date, default=datetime.date.today)
    due_date=Column(Date, nullable=False)
    return_date = Column(Date, nullable=True)
    status = Column(String(20), default="On Loan")  # On Loan / Returned
    book = relationship("Book", back_populates="loans")
    borrower = relationship("Borrower", back_populates="loans")

In [38]:
# --- Create Tables in MySQL ---
Base.metadata.create_all(engine)

2025-09-25 12:14:22,825 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-09-25 12:14:22,825 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-25 12:14:22,826 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-09-25 12:14:22,826 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-25 12:14:22,827 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-09-25 12:14:22,827 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-25 12:14:22,828 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-25 12:14:22,829 INFO sqlalchemy.engine.Engine DESCRIBE `lianelibrary`.`authors`
2025-09-25 12:14:22,829 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-25 12:14:22,831 INFO sqlalchemy.engine.Engine DESCRIBE `lianelibrary`.`books`
2025-09-25 12:14:22,831 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-25 12:14:22,831 INFO sqlalchemy.engine.Engine DESCRIBE `lianelibrary`.`borrowers`
2025-09-25 12:14:22,831 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-25 12:14:22,833 INFO sqla

In [83]:
class LibraryManegament:
    def __init__(self,session:Session):
        self.session=session


    def search_book(self,book_title:str):
        return self.session.query(Book).filter_by(title=book_title).first()

        
    
    def add_book(self,book_title:str,author_name:str,genre:str,publisher:str,language:str,copies_total=1):
        book = self.session.query(Book).filter_by(title=book_title).first()
        author = self.session.query(Author).filter_by(name=author_name).first()
        if not author:
            author = Author(name=author_name)
            self.session.add(author)
            self.session.commit()
        if not book:
            book = Book(
                title=book_title,
                author=author,
                genre=genre,
                publisher=publisher,
                copies_total=copies_total,
                copies_available=copies_total,
                language=language,
            )
  
            self.session.add(book)
            self.session.commit()
    
    def loan_book(
        self,
        borrower_name: str,
        borrower_email: str,
        book_title: str,
        borrowe_phone:str=None,
        borrowe_address:str=None,
        loan_days: int = 14  # default 2 weeks loan
    ):
    
        # --- Borrower ---
        borrower = self.session.query(Borrower).filter_by(email=borrower_email).first()
        if not borrower:
            borrower = Borrower(
                name=borrower_name,
                email=borrower_email,
                membership_date=datetime.date.today()
            )
            self.session.add(borrower)
            self.session.commit()
    
        # --- Check Book Availability ---
        book = session.query(Book).filter_by(title=book_title).first()
        if book.copies_available < 1:
            raise Exception(f"No copies available for '{book_title}'.")
    
        # --- Loan ---
        today = datetime.date.today()
        due_date = today + datetime.timedelta(days=loan_days)
    
        loan = Loan(
            book=book,
            borrower=borrower,
            loan_date=today,
            due_date=due_date,
            status="On Loan"
        )
        self.session.add(loan)

        book.copies_available -= 1
        session.commit()

    def return_book(self,book_title:str):
        book = self.session.query(Book).filter_by(title=book_title).first()
        book.copies_available +=1
        loan = session.query(Loan).filter_by(book_id=book.id).first()
        today = datetime.date.today()
        loan.return_date=today
        loan.status='Returned'
        session.commit()

    
    
    
    